In [1]:
import requests
import pandas as pd
import json
import numpy as np

In [2]:
gameName='Hitman 3'
gameNameAlt='Hitman 3'

--- here starts currency collection from ecb-------

In [3]:
# Building blocks for the URL
entrypoint = 'https://sdw-wsrest.ecb.europa.eu/service/' # Using protocol 'https'
resource = 'data'           # The resource for data queries is always'data'
flowRef ='EXR'              # Dataflow describing the data that needs to be returned, exchange rates in this case
key = 'D.SEK.EUR.SP00.A'    # Defining the dimension values, explained below

# Define the parameters
parameters = {
    'startPeriod': '2000-01-01',  # Start date of the time series
    'endPeriod': '2022-10-10'     # End of the time series
}

In [4]:
# Construct the URL: https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.CHF.EUR.SP00.A
request_url = entrypoint + resource + '/'+ flowRef + '/' + key

# Make the HTTP request again, now requesting for CSV format
response = requests.get(request_url, params=parameters, headers={'Accept': 'text/csv'})
# Response succesful? (Response code 200)
print(response)

<Response [200]>


In [5]:
open('csvs/temp.csv', 'wb').write(response.content)
currency_df = pd.read_csv('csvs/temp.csv')

In [6]:
currency_df.keys()

Index(['KEY', 'FREQ', 'CURRENCY', 'CURRENCY_DENOM', 'EXR_TYPE', 'EXR_SUFFIX',
       'TIME_PERIOD', 'OBS_VALUE', 'OBS_STATUS', 'OBS_CONF', 'OBS_PRE_BREAK',
       'OBS_COM', 'TIME_FORMAT', 'BREAKS', 'COLLECTION', 'COMPILING_ORG',
       'DISS_ORG', 'DOM_SER_IDS', 'PUBL_ECB', 'PUBL_MU', 'PUBL_PUBLIC',
       'UNIT_INDEX_BASE', 'COMPILATION', 'COVERAGE', 'DECIMALS', 'NAT_TITLE',
       'SOURCE_AGENCY', 'SOURCE_PUB', 'TITLE', 'TITLE_COMPL', 'UNIT',
       'UNIT_MULT'],
      dtype='object')

In [7]:
currency_df.tail(1)['OBS_VALUE']

5890    10.9502
Name: OBS_VALUE, dtype: float64

In [8]:
10.9502*16.69

182.75883800000003

In [9]:
ts = currency_df.filter(['TIME_PERIOD', 'OBS_VALUE'], axis=1)
# 'TIME_PERIOD' was of type 'object' (as seen in df.info). Convert it to datetime first
ts['TIME_PERIOD'] = pd.to_datetime(ts['TIME_PERIOD'])
# Set 'TIME_PERIOD' to be the index
ts = ts.set_index('TIME_PERIOD')
# Print the last 5 rows to screen
ts.tail(1)

,OBS_VALUE
TIME_PERIOD,
2022-10-10,10.9502


In [10]:
rate_sek = currency_df.tail(1)['OBS_VALUE'].values[0]
rate_sek

10.9502

--- end of currency collection -----